In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_merlin_01-building-recommender-systems-with-merlin/nvidia_logo.png" style="width: 90px; float: right;"> 

## Building Intelligent Recommender Systems with Merlin

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container. 

### Overview

Recommender Systems (RecSys) are the engine of the modern internet and the catalyst for human decisions. Building a recommendation system is challenging because it requires multiple stages (data preprocessing, offline training, item retrieval, filtering, ranking, ordering, etc.) to work together seamlessly and efficiently. The biggest challenges for new practitioners are the lack of understanding around what RecSys look like in the real world, and the gap between examples of simple models and a production-ready end-to-end recommender systems.

The figure below represents a four-stage recommender systems. This is a more complex process than only training a single model and deploying it, and it is much more realistic and closer to what's happening in the real-world recommender production systems.

![fourstage](../images/fourstages.png)

In these series of notebooks, we are going to showcase how we can deploy a four-stage recommender systems using Merlin Systems library easily on [Triton Inference Server](https://github.com/triton-inference-server/server). Let's go over the concepts in the figure briefly. 
- **Retrieval:** This is the step to narrow down millions of items into thousands of candidates. We are going to train a Two-Tower item retrieval model to retrieve the relevant top-K candidate items.
- **Filtering:** This step is to exclude the already interacted  or undesirable items from the candidate items set or to apply business logic rules. Although this is an important step, for this example we skip this step.
- **Scoring:** This is also known as ranking. Here the retrieved and filtered candidate items are being scored. We are going to train a ranking model to be able to use at our scoring step. 
- **Ordering:** At this stage, we can order the final set of items that we want to recommend to the user. Here, we’re able to align the output of the model with business needs, constraints, or criteria.

To learn more about the four-stage recommender systems, you can listen to Even Oldridge's [Moving Beyond Recommender Models talk](https://www.youtube.com/watch?v=5qjiY-kLwFY&list=PL65MqKWg6XcrdN4TJV0K1PdLhF_Uq-b43&index=7) at KDD'21 and read more [in this blog post](https://eugeneyan.com/writing/system-design-for-discovery/).

### Learning objectives
- Understanding four stages of recommender systems
- Training retrieval and ranking models with Merlin Models
- Setting up feature store and approximate nearest neighbours (ANN) search libraries
- Deploying trained models to Triton Inference Server with Merlin Systems

In addition to NVIDIA Merlin libraries and the Triton Inference Server client library, we use two external libraries in these series of examples:

- [Feast](https://docs.feast.dev/): an end-to-end open source feature store library for machine learning
- [Milvus](https://github.com/matrixji/python-milvus-server): a library for efficient similarity search and clustering of dense vectors

You can find more information about `Feast feature store` and `Milvus` libraries in the next notebook.

### Import required libraries and functions

**Compatibility:**

This notebook is developed and tested using the latest `merlin-tensorflow` container from the NVIDIA NGC catalog. To find the tag for the most recently-released container, refer to the [Merlin TensorFlow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow) page.

In [2]:
# for running this example, install the following version of the Feast library
%pip install "feast==0.18.1"

# for running this example on CPU, uncomment the following lines
# %pip install tensorflow-cpu "feast==0.18.1"
# %pip uninstall cudf

# The second notebook will use Milvus server and pymilvus, which can be installed as follows:
# %pip install milvus
# %pip install pymilvus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.2/275.2 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 48.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 70.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 266.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import nvtabular as nvt
from nvtabular.ops import Rename, Filter, Dropna, LambdaOp, Categorify, \
    TagAsUserFeatures, TagAsUserID, TagAsItemFeatures, TagAsItemID, AddMetadata

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
from merlin.datasets.ecommerce import transform_aliccp
import tensorflow as tf

# for running this example on CPU, comment out the line below
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

2023-04-10 11:56:58.434241: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")


[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so


2023-04-10 11:57:02.028040: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 11:57:04.483273: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2023-04-10 11:57:04.483365: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1637] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8192 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:06:00.0, compute capability: 7.0
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/e

[SOK INFO] Initialize finished, communication tool: horovod


In [4]:
# disable INFO and DEBUG logging everywhere
import logging

logging.disable(logging.WARNING)

In this example notebook, we will generate the synthetic train and test datasets mimicking the real [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset to build our recommender system models.

First, we define our input path and feature repo path.

In [5]:
DATA_FOLDER = os.environ.get("DATA_FOLDER", "/workspace/data/")
# set up the base dir for feature store
BASE_DIR = os.environ.get(
    "BASE_DIR", "/workspace/systems/examples/multistage_milvus/"
)

Then, we use `generate_data` utility function to generate synthetic dataset. 

In [6]:
from merlin.datasets.synthetic import generate_data

NUM_ROWS = os.environ.get("NUM_ROWS", 100_000)
train_raw, valid_raw = generate_data("aliccp-raw", int(NUM_ROWS), set_sizes=(0.7, 0.3))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


If you would like to use the real ALI-CCP dataset, you can use [get_aliccp()](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/datasets/ecommerce/aliccp/dataset.py) function instead. This function takes the raw csv files, and generate parquet files that can be directly fed to NVTabular workflow above.

### Feature Engineering with NVTabular

In [7]:
output_path = os.path.join(DATA_FOLDER, "processed_nvt")

In the following NVTabular workflow, notice that we apply the `Dropna()` Operator at the end. We add the Operator to remove rows with missing values in the final DataFrame after the preceding transformations. Although, the synthetic dataset that we generate and use in this notebook does not have null entries, you might have null entries in your `user_id` and `item_id` columns in your own custom dataset. Therefore, while applying `Dropna()` we will not be registering null `user_id_raw` and `item_id_raw` values in the feature store, and will be avoiding potential issues that can occur because of any null entries.

In [8]:
user_id_raw = ["user_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsUserFeatures()
item_id_raw = ["item_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsItemFeatures()

user_id = ["user_id"] >> Categorify(dtype="int32") >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype="int32") >> TagAsItemID()

item_features = (
    ["item_category", "item_shop", "item_brand"] >> Categorify(dtype="int32") >> TagAsItemFeatures()
)

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ] >> Categorify(dtype="int32") >> TagAsUserFeatures()
)

targets = ["click"] >> AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "target"])

outputs = user_id + item_id + item_features + user_features + user_id_raw + item_id_raw + targets

# add dropna op to filter rows with nulls
outputs = outputs >> Dropna()

Let's call `transform_aliccp` utility function to be able to perform `fit` and `transform` steps on the raw dataset applying the operators defined in the NVTabular workflow pipeline below, and also save our workflow model. After fit and transform, the processed parquet files are saved to output_path.

In [9]:
transform_aliccp(
    (train_raw, valid_raw), output_path, nvt_workflow=outputs, workflow_name="workflow"
)

### Training a Retrieval Model with Two-Tower Model

We start with the offline candidate retrieval stage. We are going to train a Two-Tower model for item retrieval. To learn more about the Two-tower model you can visit [05-Retrieval-Model.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb).

#### Feature Engineering with NVTabular

We are going to process our raw categorical features by encoding them using `Categorify()` operator and tag the features with `user` or `item` tags in the schema file. To learn more about [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular) and the schema object visit this example [notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb) in the Merlin Models repo.

Define a new output path to store the filtered datasets and schema files.

In [10]:
output_path2 = os.path.join(DATA_FOLDER, "processed/retrieval")

In [11]:
train_tt = Dataset(os.path.join(output_path, "train", "*.parquet"))
valid_tt = Dataset(os.path.join(output_path, "valid", "*.parquet"))

We select only positive interaction rows where `click==1` in the dataset with `Filter()` operator.

In [12]:
inputs = train_tt.schema.column_names
outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

workflow2 = nvt.Workflow(outputs)

workflow2.fit(train_tt)

workflow2.transform(train_tt).to_parquet(
    output_path=os.path.join(output_path2, "train")
)

workflow2.transform(valid_tt).to_parquet(
    output_path=os.path.join(output_path2, "valid")
)

NVTabular exported the schema file, `schema.pbtxt` a protobuf text file, of our processed dataset. To learn more about the schema object and schema file you can explore [02-Merlin-Models-and-NVTabular-integration.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb) notebook.

**Read filtered parquet files as Dataset objects.**

In [13]:
train_tt = Dataset(os.path.join(output_path2, "train", "*.parquet"), part_size="500MB")
valid_tt = Dataset(os.path.join(output_path2, "valid", "*.parquet"), part_size="500MB")

In [14]:
schema = train_tt.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER]).without(['user_id_raw', 'item_id_raw', 'click'])
train_tt.schema = schema
valid_tt.schema = schema

In [15]:
model_tt = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),
    samplers=[mm.InBatchSampler()],
    embedding_options=mm.EmbeddingOptions(infer_embedding_sizes=True),
)

In [16]:
model_tt.compile(
    optimizer="adam",
    run_eagerly=False,
    loss="categorical_crossentropy",
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)],
)
model_tt.fit(train_tt, validation_data=valid_tt, batch_size=1024 * 8, epochs=1)

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


5/5 [==============================] - 20s 697ms/step - loss: 8.9095 - recall_at_10: 0.0172 - ndcg_at_10: 0.0127 - regularization_loss: 0.0000e+00 - loss_batch: 8.5680 - val_loss: 8.9080 - val_recall_at_10: 0.0249 - val_ndcg_at_10: 0.0206 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 8.8060


### Exporting query (user) model

We export the query tower to use it later during the model deployment stage with Merlin Systems.

In [17]:
query_tower = model_tt.retrieval_block.query_block()
query_tower.save(os.path.join(BASE_DIR, "query_tower"))

### Training a Ranking Model with DLRM

Now we will move onto training an offline ranking model. This ranking model will be used for scoring our retrieved items.

Read processed parquet files. We use the `schema` object to define our model.

In [18]:
# define train and valid dataset objects
train = Dataset(os.path.join(output_path, "train", "*.parquet"), part_size="500MB")
valid = Dataset(os.path.join(output_path, "valid", "*.parquet"), part_size="500MB")

# define schema object
schema = train.schema.without(['user_id_raw', 'item_id_raw'])

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [19]:
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'click'

Deep Learning Recommendation Model [(DLRM)](https://arxiv.org/abs/1906.00091) architecture is a popular neural network model originally proposed by Facebook in 2019. The model was introduced as a personalization deep learning model that uses embeddings to process sparse features that represent categorical data and a multilayer perceptron (MLP) to process dense features, then interacts these features explicitly using the statistical techniques proposed in [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5694074). To learn more about DLRM architetcture please visit `Exploring-different-models` [notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/04-Exporting-ranking-models.ipynb) in the Merlin Models GH repo.

In [20]:
model = mm.DLRMModel(
    schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
)

In [21]:
model.compile(optimizer="adam", run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=16 * 1024)

5/5 [==============================] - 9s 511ms/step - loss: 0.6932 - auc: 0.4975 - regularization_loss: 0.0000e+00 - loss_batch: 0.6932 - val_loss: 0.6932 - val_auc: 0.5001 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6932


Let's save our DLRM model to be able to load back at the deployment stage. 

In [22]:
model.save(os.path.join(BASE_DIR, "dlrm"))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In the following cells we are going to export the required user and item features files, and save the query (user) tower model and item embeddings to disk. If you want to read more about exporting retrieval models, please visit [05-Retrieval-Model.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb) notebook in Merlin Models library repo.

### Set up a feature store with Feast

Before we move onto the next step, we need to create a Feast feature repository. [Feast](https://feast.dev/) is an end-to-end open source feature store for machine learning. Feast (Feature Store) is a customizable operational data system that re-uses existing infrastructure to manage and serve machine learning features to real-time models.

We will create the feature repo in the current working directory, which is `BASE_DIR` for us.

In [23]:
#!pip install feast==0.18.1

In [24]:
!rm -rf $BASE_DIR/feature_repo
!cd $BASE_DIR && feast init feature_repo

Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage

Creating a new Feast repository in /workspace/systems/examples/multistage_milvus/feature_repo.



You should be seeing a message like <i>Creating a new Feast repository in ... </i> printed out above. Now, navigate to the `feature_repo` folder and remove the demo parquet file created by default, and `examples.py` file.

In [25]:
feature_repo_path = os.path.join(BASE_DIR, "feature_repo")
if os.path.exists(f"{feature_repo_path}/example.py"):
    os.remove(f"{feature_repo_path}/example.py")
if os.path.exists(f"{feature_repo_path}/data/driver_stats.parquet"):
    os.remove(f"{feature_repo_path}/data/driver_stats.parquet")

### Exporting user and item features

In [26]:
from merlin.models.utils.dataset import unique_rows_by_features

user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID)
    .compute()
    .reset_index(drop=True)
)

In [27]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories,user_id_raw
0,1,1,1,1,1,1,1,1,1,1,1,1,6
1,2,2,1,1,1,1,1,1,1,2,2,2,9
2,3,3,1,1,1,1,1,1,1,3,3,3,7
3,4,4,1,1,1,1,1,1,1,4,4,4,5
4,5,5,1,1,1,1,1,1,1,5,5,5,8


We will artificially add `datetime` and `created` timestamp columns to our user_features dataframe. This required by Feast to track the user-item features and their creation time and to determine which version to use when we query Feast.

In [28]:
from datetime import datetime

user_features["datetime"] = datetime.now()
user_features["datetime"] = user_features["datetime"].astype("datetime64[ns]")
user_features["created"] = datetime.now()
user_features["created"] = user_features["created"].astype("datetime64[ns]")

In [29]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories,user_id_raw,datetime,created
0,1,1,1,1,1,1,1,1,1,1,1,1,6,2023-04-10 11:59:03.409313,2023-04-10 11:59:03.410951
1,2,2,1,1,1,1,1,1,1,2,2,2,9,2023-04-10 11:59:03.409313,2023-04-10 11:59:03.410951
2,3,3,1,1,1,1,1,1,1,3,3,3,7,2023-04-10 11:59:03.409313,2023-04-10 11:59:03.410951
3,4,4,1,1,1,1,1,1,1,4,4,4,5,2023-04-10 11:59:03.409313,2023-04-10 11:59:03.410951
4,5,5,1,1,1,1,1,1,1,5,5,5,8,2023-04-10 11:59:03.409313,2023-04-10 11:59:03.410951


In [30]:
user_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "user_features.parquet")
)

In [31]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
    .compute()
    .reset_index(drop=True)
)

In [32]:
item_features["datetime"] = datetime.now()
item_features["datetime"] = item_features["datetime"].astype("datetime64[ns]")
item_features["created"] = datetime.now()
item_features["created"] = item_features["created"].astype("datetime64[ns]")

In [33]:
item_features.head()

,item_id,item_category,item_shop,item_brand,item_id_raw,datetime,created
0,1,1,1,1,8,2023-04-10 11:59:03.559757,2023-04-10 11:59:03.561342
1,2,2,2,2,6,2023-04-10 11:59:03.559757,2023-04-10 11:59:03.561342
2,3,3,3,3,7,2023-04-10 11:59:03.559757,2023-04-10 11:59:03.561342
3,4,4,4,4,5,2023-04-10 11:59:03.559757,2023-04-10 11:59:03.561342
4,5,5,5,5,10,2023-04-10 11:59:03.559757,2023-04-10 11:59:03.561342


In [34]:
# save to disk
item_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "item_features.parquet")
)

### Extract and save Item embeddings

In [35]:
item_embs = model_tt.item_embeddings(
    Dataset(item_features, schema=schema), batch_size=1024
)
item_embs_df = item_embs.compute(scheduler="synchronous")

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [36]:
# select only item_id together with embedding columns
item_embeddings = item_embs_df.drop(
    columns=["item_category", "item_shop", "item_brand"]
)

In [37]:
item_embeddings.head()

,item_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,-0.008910,0.014080,-0.014629,-0.039765,0.017970,0.033510,-0.004685,-0.023115,0.008598,...,0.018147,-0.015784,0.000822,0.006560,-0.021421,-0.004947,0.018377,0.002227,-0.017367,-0.005241
1,2,-0.020520,0.033888,-0.063310,0.003641,-0.006738,-0.076620,-0.028552,0.005106,-0.001853,...,-0.021206,0.014955,0.008985,0.012912,-0.007857,0.022142,0.003478,0.025290,0.084739,0.068159
2,3,-0.022046,-0.037312,-0.087176,-0.016671,0.036148,-0.040209,-0.008577,0.015657,0.023547,...,0.006433,0.026370,0.002353,0.039969,-0.034152,-0.008681,0.020700,0.029407,0.044596,0.065746
3,4,-0.016704,-0.008492,-0.027469,-0.027047,0.011672,-0.033592,-0.019881,-0.017269,0.040512,...,0.029454,-0.017885,0.005354,-0.001075,-0.042389,-0.036338,0.033087,0.029585,0.009247,0.028567
4,5,-0.003928,0.005124,-0.020727,-0.023249,-0.005649,-0.008514,-0.007825,0.005726,0.013559,...,0.019905,-0.009648,0.004087,0.011984,-0.007550,-0.006617,0.002931,0.026207,0.001347,0.012543


In [38]:
# save to disk
item_embeddings.to_parquet(os.path.join(BASE_DIR, "item_embeddings.parquet"))

In [39]:
user_embs = model_tt.query_embeddings(
    Dataset(user_features, schema=schema), batch_size=1024
)
user_embs_df = user_embs.compute(scheduler="synchronous")

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


In [40]:
user_embs_df.columns

Index(['user_id', 'user_shops', 'user_profile', 'user_group', 'user_gender',
       'user_age', 'user_consumption_2', 'user_is_occupied', 'user_geography',
       'user_intentions', 'user_brands', 'user_categories', '0', '1', '2', '3',
       '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
       '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52',
       '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63'],
      dtype='object')

In [41]:
# select only item_id together with embedding columns
user_embeddings = user_embs_df.drop(
    columns=['user_shops', 'user_profile', 'user_group', 'user_gender',
       'user_age', 'user_consumption_2', 'user_is_occupied', 'user_geography',
       'user_intentions', 'user_brands', 'user_categories']
)

In [42]:
# save to disk
user_embeddings.to_parquet(os.path.join(BASE_DIR, "user_embeddings.parquet"))

### Create feature definitions 

Now we will create our user and item features definitions in the user_features.py and item_features.py files and save these files in the feature_repo.

In [43]:
file = open(os.path.join(BASE_DIR, "feature_repo/", "user_features.py"), "w")
file.write(
    """
from google.protobuf.duration_pb2 import Duration
import datetime
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

user_features = FileSource(
    path="{}",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

user_raw = Entity(name="user_id_raw", value_type=ValueType.INT32, description="user id raw",)

user_features_view = FeatureView(
    name="user_features",
    entities=["user_id_raw"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="user_shops", dtype=ValueType.INT32),
        Feature(name="user_profile", dtype=ValueType.INT32),
        Feature(name="user_group", dtype=ValueType.INT32),
        Feature(name="user_gender", dtype=ValueType.INT32),
        Feature(name="user_age", dtype=ValueType.INT32),
        Feature(name="user_consumption_2", dtype=ValueType.INT32),
        Feature(name="user_is_occupied", dtype=ValueType.INT32),
        Feature(name="user_geography", dtype=ValueType.INT32),
        Feature(name="user_intentions", dtype=ValueType.INT32),
        Feature(name="user_brands", dtype=ValueType.INT32),
        Feature(name="user_categories", dtype=ValueType.INT32),
        Feature(name="user_id", dtype=ValueType.INT32),
    ],
    online=True,
    input=user_features,
    tags=dict(),
)
""".format(
        os.path.join(BASE_DIR, "feature_repo/data/", "user_features.parquet")
    )
)
file.close()

In [44]:
with open(os.path.join(BASE_DIR, "feature_repo/", "item_features.py"), "w") as f:
    f.write(
        """
from google.protobuf.duration_pb2 import Duration
import datetime
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

item_features = FileSource(
    path="{}",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

item = Entity(name="item_id", value_type=ValueType.INT32, description="item id",)

item_features_view = FeatureView(
    name="item_features",
    entities=["item_id"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="item_category", dtype=ValueType.INT32),
        Feature(name="item_shop", dtype=ValueType.INT32),
        Feature(name="item_brand", dtype=ValueType.INT32),
        Feature(name="item_id_raw", dtype=ValueType.INT32),
    ],
    online=True,
    input=item_features,
    tags=dict(),
)
""".format(
            os.path.join(BASE_DIR, "feature_repo/data/", "item_features.parquet")
        )
    )
file.close()

Let's checkout our Feast feature repository structure.

In [45]:
# install seedir
!pip install seedir

     |████████████████████████████████| 111 kB 21.5 MB/s eta 0:00:01


In [46]:
import seedir as sd

feature_repo_path = os.path.join(BASE_DIR, "feature_repo")
sd.seedir(
    feature_repo_path,
    style="lines",
    itemlimit=10,
    depthlimit=3,
    exclude_folders=".ipynb_checkpoints",
    sort=True,
)

feature_repo/
├─__init__.py
├─data/
│ ├─item_features.parquet
│ └─user_features.parquet
├─feature_store.yaml
├─item_features.py
└─user_features.py


### Next Steps
We trained and exported our ranking and retrieval models and NVTabular workflows. In the next step, we will learn how to deploy our trained models into [Triton Inference Server (TIS)](https://github.com/triton-inference-server/server) with Merlin Systems library.

For the next step, move on to the `02-Deploying-multi-stage-Recsys-with-Merlin-Systems-Milvus.ipynb` notebook to deploy our saved models as an ensemble to TIS and obtain prediction results for a given request.